In [ ]:
using DataFrames, Distributions, ForwardDiff

In [ ]:
# Stochastic gradient descent, mixed logit.

df = readtable("../data/parsed_model_australia.txt", separator = ' ', header = false)

a, b = size(df)

const n_individuals = a
const n_alternatives = 4
const n_parameters = b
const n_simulations = 1000
const simulated_b = 5

srand(123456)

rand_contdist(Dist::Distribution) = quantile(Dist, rand())

head(df)

In [ ]:
mixed_logit = zeros(n_individuals, n_simulations)

function simulate()
    for i = 1:n_individuals, j = 1:n_simulations
        mixed_logit[i, j] = rand()
    end
end

simulate()

In [ ]:
function individual(θ::Vector, i::Int64)
    m, n = size(df)
    choice = df[i, 1][1]
    alternatives = collect(1:n_alternatives)
    splice!(alternatives, choice)
    
    function utility(β::Vector, k::Int64)
        temp = Float64[]
        k += 1
        while k <= n_parameters
            push!(temp, df[i, k])
            k += n_alternatives
        end
        return dot(temp, β)
    end
    
    function construct(γ::Vector, θ::Vector)
        return θ[1]+θ[2]*γ[1]
    end
    
    function probability(θ::Vector)
        logit = 0.0
        t = 0.0
        for k = 1:n_simulations
            β = []
            for j = 1:simulated_b-1
                push!(β, θ[j])
            end
            γ = [mixed_logit[i, k]]
            push!(β, construct(γ, θ[simulated_b:simulated_b+1]))
            for j = simulated_b+2:length(θ)
                push!(β, θ[j])
            end 
            c = utility(β, choice)
            for alternative in alternatives
                t += exp(utility(β, alternative)-c)
            end
            logit += 1/(1+t)
        end
        return logit/n_simulations
    end
    
    return probability
end

In [ ]:
function f(β::Vector)
    model = 0.0
    i = 1
    while i <= n_individuals
        logit = individual(β, i)
        model += log(logit(β))
        i += 1
    end
    return -model/n_individuals
end

In [ ]:
function g(x::Vector, i::Int64, j::Int64)
    t = zeros(length(x))
    for k = i:j
        logit = individual(x, k)
        t += (1/logit(x))*ForwardDiff.gradient(logit, x)
    end
    return t
end

function g!(x::Vector, i::Int64, j::Int64, storage::Vector)
    s = g(x, i, j)
    storage[1:length(s)] = s[1:length(s)]
end

In [ ]:
function shuffle!(df::DataFrame)
    for i = size(df, 1):-1:2
        j = rand(1:i)
        df[i, :], df[j, :] = df[j, :], df[i, :]
    end
end

In [ ]:
function sgd(g!::Function, ω0::Vector, η::Float64, batch_size::Int64 = 1, tol::Float64 = 1e-6)
    k = 1
    ω = ω0
    n = length(ω)
    dfx = ones(n)
    nmax = 1000 # epochs
    change = 1.0
    while change > tol && k < nmax
        shuffle!(df)
        x = ω
        for i = 1:batch_size:n_individuals
            ω -= η*g!(x, i, batch_size, dfx)
        end
        change = abs(f(ω)-f(x))
        k += 1
    end
    return ω, k, f(ω)
end

In [ ]:
sgd(g!, zeros(7), 0.001)